# Setup

In [1]:
%load_ext autoreload

In [11]:
%autoreload 2

import copy
import functools
import itertools
import logging
import re
import time
import yaml
from collections import Counter

import humanize
from pymicro.file import file_utils
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import scipy as sp
from numpy import ndarray
from matplotlib import pyplot as plt, patches, cm
from sklearn import metrics as met
from progressbar import progressbar as pbar
from tensorflow.keras import layers
from sklearn import metrics, preprocessing
from sklearn import model_selection
import skimage
from skimage import measure, io
import pandas as pd

import tomo2seg.data as tomo2seg_data
from tomo2seg.data import ModelPaths, Volume, EstimationVolume, SetPartition
from tomo2seg.volume_img_segm import VolumeImgSegmSequence
from tomo2seg import viz
from tomo2seg.logger import logger

In [3]:
from tomo2seg.data import VOLUME_PRECIPITATES_V1 as VOL_NAME_VERSION
logger.debug(f"{VOL_NAME_VERSION=}")

In [4]:
logger.setLevel(logging.DEBUG)

In [5]:
random_state = 42
random_state = np.random.RandomState(random_state)

# Load from the disk

In [9]:
# Metadata/paths objects

## Volume
volume = Volume.with_check(
    volume_name := VOL_NAME_VERSION[0], 
    volume_version := VOL_NAME_VERSION[1]
)
logger.info(f"{volume=}")

hst_read = lambda x: functools.partial(
    # from pymicro
    file_utils.HST_read,
    # pre-loaded kwargs
    autoparse_filename=False,  # the file names are not properly formatted
    data_type=volume.metadata.dtype,
    dims=volume.metadata.dimensions,
    verbose=True,
)(str(x))  # it doesn't accept paths...

blobs_path_prefix = str(volume.versioned_labels_path("original"))[:-4]
logger.debug(f"{blobs_path_prefix=}")

[2020-11-10::16:54:18.100] tomo2seg :: DEBUG :: {data.py:with_check:201}
vol=Volume(name='PA66GF30', version='v1', _metadata=None)

[2020-11-10::16:54:18.103] tomo2seg :: ERROR :: {data.py:with_check:219}
Missing file: /home/users/jcasagrande/projects/tomo2seg/data/PA66GF30.v1/PA66GF30.v1.labels.raw

[2020-11-10::16:54:18.104] tomo2seg :: WARNING :: {data.py:with_check:223}
Missing file: /home/users/jcasagrande/projects/tomo2seg/data/PA66GF30.v1/PA66GF30.v1.weights.raw

[2020-11-10::16:54:18.106] tomo2seg :: DEBUG :: {data.py:metadata:158}
Loading metadata from `/home/users/jcasagrande/projects/tomo2seg/data/PA66GF30.v1/PA66GF30.v1.metadata.yml`.

[2020-11-10::16:54:18.120] tomo2seg :: INFO :: {<ipython-input-9-4fefebe63d9f>:<module>:008}
volume=Volume(name='PA66GF30', version='v1', _metadata=Volume.Metadata(dimensions=[1300, 1040, 1900], dtype='uint8', labels=[0, 1, 2], labels_names={0: 'matrix', 1: 'fiber', 2: 'porosity'}, set_partitions={'train': {'x_range': [0, 1299], 'y_range': [0

# Porosity blobs

## z-axis 2d blobs

**Note about the filenames here**

Search for (Ctrl + F) `[filenames]` in the script `compute_2d_blob_props.py` to see where they come from.

In [12]:
filename = f"{blobs_path_prefix}.porosity.x-blobs.csv"
logger.debug(f"{filename=}")

blobsdf = pd.read_csv(filename)
logger.debug(f"{blobsdf.shape}")

[2020-11-10::16:59:45.091] tomo2seg :: DEBUG :: {<ipython-input-12-9e0718de7087>:<module>:002}
filename='/home/users/jcasagrande/projects/tomo2seg/data/PA66GF30.v1/PA66GF30.v1.labels-original.porosity.x-blobs.csv'

[2020-11-10::16:59:46.844] tomo2seg :: DEBUG :: {<ipython-input-12-9e0718de7087>:<module>:005}
(1147786, 23)

